<a href="https://colab.research.google.com/github/MohitSolunke/HybridSearch-RAG-with-Langchain/blob/main/Hybrid_Search_RAG_with_Langchain_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [14]:
api_key = "e19db484-fa50-4176-882d-55832b7f023b"

In [15]:
!pip install langchain_community

In [16]:

from langchain_community.retrievers import PineconeHybridSearchRetriever

In [17]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name  = 'hybrid-search-langchain-pineconevectiordb'

##Intialize the pinecone client

pc = Pinecone(api_key=api_key)

## Create the index
if index_name not in pc.list_indexes().names():
  pc.create_index(name=index_name, dimension=384,  ##Dimention of Dense Vector
                  metric="dotproduct" ,## Sparse values supported only for dotproducts
                  spec = ServerlessSpec(cloud='aws', region='us-east-1')
                  )


In [18]:
index = pc.Index(index_name)
index

In [19]:
!pip install langchain_huggingface

In [21]:
## Vector Embedding and Sparse Matrix
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
from pinecone_text.sparse import BM25Encoder ## These Encoder uses TF/IDF tech for making vector by default

bm25_encoder = BM25Encoder().default()
bm25_encoder


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
## We create the sparse matrix for below sentences
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited London",
    "In 2020, I visited Berlin",

]
## Applying tfidf value on these sentences
bm25_encoder.fit(sentences)

## Store the values to a json file
bm25_encoder.dump("bm25_encoder.json")

## Load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_encoder.json")

  0%|          | 0/4 [00:00<?, ?it/s]

In [26]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,
                                          sparse_encoder=bm25_encoder,
                                          index=index)

In [27]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7c32f0385000>, index=<pinecone.data.index.Index object at 0x7c342c99da20>)

In [28]:
retriever.add_texts([
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited London",
    "In 2020, I visited Berlin",

])

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
retriever.invoke("what city did i recent last ")

[Document(page_content='In 2021, I visited London'),
 Document(page_content='In 2020, I visited Berlin'),
 Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2023, I visited Paris')]